In [2]:
import pymysql
def retrieve_review():
    connection = pymysql.connect(
        host='localhost',
        user='root',
        password='1234',
        database='shoutreview',
        autocommit=True
    )
    
    cursor = connection.cursor()
    # Query to extract reviews
    query = "select r.id, r.review from review r where r.sentiment is NULL"
    cursor.execute(query)
    reviews = cursor.fetchall()
    
    # Convert the reviews from tuple format
    reviews = [review for review in reviews]
    
    # Close connection
    # connection.close()
    return reviews
retrieve_review()

[]

In [3]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def preprocess_review(text):
    # Lowercasing
    text = text.lower()
    # Remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenization
    tokens = text.split()
    # Remove stop words
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    # Join tokens back to string
    return ' '.join(tokens)

def get_cleaned_reviews(reviews):
    cleaned_reviews = [preprocess_review(review[1]) for review in reviews]
    return reviews, cleaned_reviews
get_cleaned_reviews(retrieve_review())

[nltk_data] Downloading package stopwords to C:\Users\Kriti
[nltk_data]     Srivastava\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


([], [])

In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

def get_sentiment(cleaned_reviews):
    sia = SentimentIntensityAnalyzer()
    
    # Analyze sentiment
    sentiments = [sia.polarity_scores(review) for review in get_cleaned_reviews(retrieve_review())[1]]
    
    # Print first review and its sentiment
    #print(sentiments[0])
    reviews= [review[0] for review in get_cleaned_reviews(retrieve_review())[0]]
    return sentiments, reviews
get_sentiment(get_cleaned_reviews(retrieve_review())[1])

[nltk_data] Downloading package vader_lexicon to C:\Users\Kriti
[nltk_data]     Srivastava\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


([], [])

In [5]:
def classify_sentiment(score):
    if score > 0:
        return 'positive'
    elif score == 0:
        return 'neutral'
    else:
        return 'negative'

In [7]:
from flask import Flask, request, jsonify
from matplotlib import pyplot as plt
# import pymysql

# Initialize Flask app
app = Flask(__name__)

connection = pymysql.connect(
        host='localhost',
        user='root',
        password='1234',
        database='shoutreview',
        autocommit=True)
cursor = connection.cursor()

@app.route('/analyze', methods=['POST'])
def analyze_sentiment():  
    try:
        # connection = pymysql.connect(
        # host='localhost',
        # user='root',
        # password='1234',
        # database='shoutreview',
        # autocommit=True)
        # cursor = connection.cursor()
        
        review= retrieve_review()
        review= get_cleaned_reviews(review)
        sentiments, review= get_sentiment(review)
        sentiment_labels = [classify_sentiment(sentiment['compound']) for sentiment in sentiments]

        for sentiment, review_id in zip(sentiment_labels, review):
            insert_query = "UPDATE review SET sentiment = %s WHERE id = %s"
            cursor.execute(insert_query, (sentiment, review_id))
        connection.commit()
    except Exception as e:
        return jsonify({'error': str(e)}), 500
    finally:
        connection.close()
    
    # Return the sentiment analysis result
    return jsonify({
        'message' : 'Done',
        'sentiment' : sentiment_labels[0]
    })

@app.get("/genre")
def genre_analysis():
    try:
        query= "select movie.genre from user_favorites join movie on user_favorites.movie_id = movie.id"
        cursor.execute(query)
        genres = cursor.fetchall()
        genres = [genre[0] for genre in genres]
        genre_count = {}
        for genre in genres:
            if genre in genre_count:
                genre_count[genre] += 1
            else:
                genre_count[genre] = 1
        plt.figure(figsize=(10, 8))
        plt.pie(list(genre_count.values()), 
            labels=list(genre_count.keys()), 
            autopct='%1.1f%%')
        plt.title('Distribution of Movie Genres in User Favorites Tab')
        from io import BytesIO
        import base64
        buf = BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        plot_url = base64.b64encode(buf.getvalue()).decode('utf-8')
        plt.close()
    
        # Return the plot as part of the response
        return f'<img src="data:image/png;base64,{plot_url}">'
    except Exception as e:
        return jsonify({'error': str(e)}), 500
    finally:
        connection.close()

# Run the app
if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Feb/2025 23:45:22] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [22/Feb/2025 23:45:28] "GET /genre HTTP/1.1" 200 -
